In [1]:
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch

In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))

In [80]:
import pandas as pd

df_com = pd.read_csv("complaint1700.csv")
df_ncom = pd.read_csv("noncomplaint1700.csv")
df_tst = pd.read_csv("test_data.csv")

In [81]:
import re
lemmatizer = WordNetLemmatizer()
def text_prep(text):
  corp = str(text).lower()
  corp = re.sub(r'[^a-zA-Z]+',' ',corp)
  corp = re.sub(r'  ',' ',corp).strip()
  tokens = word_tokenize(corp)
  words = [t for t in tokens if t not in stop_words]
  return " ".join(words)

In [82]:
df_com['clean_text'] = df_com['tweet'].apply(lambda x: text_prep(x))

In [83]:
df_ncom['clean_text'] = df_ncom['tweet'].apply(lambda x: text_prep(x))

In [84]:
df_tst['clean_text'] = df_tst['tweet'].apply(lambda x: text_prep(x))

In [85]:
df_com["sentiment"] = "complaint"

In [86]:
df_com["sentiment-label"] = "1"

In [87]:
df_ncom["sentiment"] = "non-complaint"

In [88]:
df_ncom["sentiment-label"] = "0"

In [89]:
train_merge_df = pd.concat([df_com,df_ncom],ignore_index = True).sample(frac=1).reset_index(drop=True)

In [90]:
train_merge_df

,id,airline,tweet,clean_text,sentiment,sentiment-label
0,36755,American,Trying to book flight on @AmericanAir &amp; th...,trying book flight americanair amp website apo...,complaint,1
1,38453,JetBlue,@JetBlue You were my favorite airline but this...,jetblue favorite airline news baggage fees cut...,complaint,1
2,118259,United,Yeah @united Get UA 89 off he ground! This is ...,yeah united get ua ground ridiculous,complaint,1
3,172637,United,You suck big time @United screw all your crews...,suck big time united screw crews unprofessiona...,complaint,1
4,84016,Delta,@DeltaAssist I am waiting for your action you ...,deltaassist waiting action promised back augus...,complaint,1
...,...,...,...,...,...,...
3395,15524,United,@DanielleAllan12 @nbcwashington @united I vote...,danielleallan nbcwashington united vote space ...,non-complaint,0
3396,21822,American,@AmericanAir is the worst! My flight to Chicag...,americanair worst flight chicago cancelled due...,complaint,1
3397,69581,JetBlue,Serious issue with @JetBlue flight from NYC to...,serious issue jetblue flight nyc lax check sis...,complaint,1
3398,16829,United,@GlobeTrotScott @Delta @AmericanAir I think @u...,globetrotscott delta americanair think united ...,non-complaint,0


In [157]:
texts = []
labels = []
texts = train_merge_df["clean_text"]
#texts = str(texts)
labels = train_merge_df["sentiment-label"]

In [158]:
len(texts)

3400

In [159]:
texts[511]

'wi fi available southwestair flight ho hum wait watch nbcparenthood happydance notissues unprepared'

In [160]:
len(labels)

3400

In [161]:
import numpy as np
labels = np.array(labels)
labels = labels.astype(float) 
labels

array([1., 1., 1., ..., 1., 0., 0.])

In [147]:
from sklearn.model_selection import train_test_split
train_texts,val_texts, train_labels,val_labels = train_test_split(texts,labels,test_size = 0.3, random_state = 42)

In [166]:
new_val = [f"{i+0}: {text}" for i, text in enumerate(val_texts)]

In [197]:
new_val = str(new_val)
new_val

"['0: miss skymall travel southwestair http co fxg zzru', '1: americanair preferred seating either sell ticket give seat check lax waste everyone time', '2: flight held inbound people miss connection support decision props united logical', '3: americanair hour phone trying get back ticket never used employees tell get back maybe follow', '4: getting home late chicago left light photo ccmotorcycle luv southwestair http co kxaotri en', '5: americanair redeem frequent flier miles mile saver ones cancel get miles back', '6: love virginamerica plane named screw let', '7: rolandsmartin rted lost bag couple years ago united tweeted amp came hours americanair', '8: united fly ohare ops waiting hrs gate ua', '9: new virginamerica site screwed drop downs scroll new interface hard work fix guys giving', '10: guys virginamerica switched almonds roasted chickpeas snack boxes still know think', '11: walkofftheearth sarahblackwood kicked united flight last night son crying shot sarah times', '12: tha

In [149]:
val_labels[511]

0.0

In [155]:
train_labels

array([0., 0., 0., ..., 0., 1., 0.])

In [162]:
new_train = [f"{i+0}: {text}" for i, text in enumerate(train_texts)]

In [164]:
new_train

['0: last road trip semester gonuwhockey wait get back board jetblue bos rainy morning pit gonu',
 '1: virginamerica whatever happened status match program going switch business',
 '2: nancyleegrahn united nhlblackhawks shrinershosp consider source brady probably thinks charity sad',
 '3: jensenackles americanair patient know wait see little jj going daddy girl sure',
 '4: cregeta united united response delayed like flights help nepal nepalese get relief supplies soon please',
 '5: miss united',
 '6: yesicandice reading tweets sorry family loss also cancelled flight americanair next month gt',
 '7: deltaassist plus sitting comfort premium snacks offered poor experience today poor',
 '8: never fails americanair miami delayed deplaned xs due mechanical safety first crew annoyed agent overwhelmed',
 '9: united safety video wilhelm scream',
 '10: alaskaair americanair offering flexible travel due weds storm worst travel day combo http co hqpucgv p',
 '11: currently jetblue flying back new 

In [183]:
new_train[144]

'144: americanair ruined day'

In [169]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
label_encoder = LabelEncoder()

encoded_labels = label_encoder.fit_transform(labels)
encoded_labels

labels_tensor = torch.Tensor(encoded_labels)
len(labels_tensor)

tokenized_inputs =tokenizer(texts,truncation = True,padding = True,max_length = 256,return_tensors = 'pt')

len(tokenized_inputs['input_ids'])

In [170]:
import torch
from torch.utils.data import Dataset, DataLoader

In [171]:
len(texts)

3400

In [172]:
len(labels)

3400

In [173]:
labels.dtype

dtype('float64')

In [184]:
max_length = 64
class CustomDataset(Dataset):
    def __init__(self,texts,labels,tokenizer,max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        inputs = self.tokenizer(text, truncation = True, padding= 'max_length', max_length = self.max_length, return_tensors = "pt")
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        
        return {
                'input_ids' : input_ids.squeeze(),
                'attention_mask' : attention_mask.squeeze(),
                'label' : torch.tensor(label, dtype= torch.long)
        }

In [185]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_length = 64 
train_dataset = CustomDataset(new_train,train_labels, tokenizer, max_length)
val_dataset = CustomDataset(new_val,val_labels, tokenizer, max_length)

len(tokenized_inputs['attention_mask'])

In [186]:
import torch
from torch.utils.data import Dataset, DataLoader
batch_size = 62
dataloader_1 = DataLoader(train_dataset,batch_size = batch_size, shuffle = True)
dataloader_2 = DataLoader(val_dataset,batch_size = batch_size, shuffle = False)

In [187]:
dataloader_1

In [188]:
from torch import nn, optim

In [189]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [190]:
optimizer = AdamW(model.parameters(), lr = 5e-5)
criterion = nn.CrossEntropyLoss()

In [191]:
train_dataloader

In [193]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()

    for batch in dataloader_1:
        input_ids = batch["input_ids"]
        attention_mask = batch['attention_mask']
        stacked_mask = torch.stack([input_ids,attention_mask], dim=0)
        labels = batch['label']

        outputs = model(input_ids, attention_mask= attention_mask)
        logits = outputs.logits

        criterion = torch.nn.CrossEntropyLoss()
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {loss.item()}")

Epoch 1/3 - Loss: 0.6728454232215881
Epoch 2/3 - Loss: 0.47617673873901367
Epoch 3/3 - Loss: 0.27804213762283325


In [204]:
from sklearn.metrics import accuracy_score

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in dataloader_2:
        input_ids = batch["input_ids"]
        attention_mask = batch['attention_mask']
        stacked_mask = torch.stack([input_ids,attention_mask], dim=0)
        labels = batch['label']
        
        outputs = model(input_ids, attention_mask = attention_mask)

        logits = outputs.logits

        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())

true_labels = val_labels
accuracy = accuracy_score(true_labels,predictions)
print("Accuracy: ",accuracy * 100)

Accuracy:  74.01960784313727


In [206]:
text = "SouthwestAir get your damn act together. Don't announce we will be leaving on time after being delayed then delaying us again!"
inputs = tokenizer(text,return_tensors ="pt")
output = model(**inputs)
predictions = torch.argmax(output.logits,dim =1).item()

print(predictions)

1


In [207]:
save_path = 'C:/Users/u387850/Documents/CnNC'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('C:/Users/u387850/Documents/CnNC\\tokenizer_config.json',
 'C:/Users/u387850/Documents/CnNC\\special_tokens_map.json',
 'C:/Users/u387850/Documents/CnNC\\vocab.txt',
 'C:/Users/u387850/Documents/CnNC\\added_tokens.json')

In [208]:
print(f"Model saved at: {save_path}")

Model saved at: C:/Users/u387850/Documents/CnNC


In [48]:
save_path = 'C:/Users/u387850/Documents/CnNC'
loaded_model = BertForSequenceClassification.from_pretrained(save_path)
loaded_tokenizer = BertTokenizer.from_pretrained(save_path)

In [ ]:
inputs = loaded_tokenizer(train_texts, return_tensors = "pt", truncation = True,padding = True)

with torch.no_grad():
    outputs = loaded_model(input_ids, attention_mask= attention_mask)
    logits = outputs.logits
    criterion = torch.nn.CrossEntropyLoss()
    loss = criterion(logits, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


